In [1]:
import serial
import time
from collections import deque
import numpy as np

RMID = 183          # RMID
TMID = 184          # TMID
ID = 1              # ID
PID_MAINBC = 10     # PID (MAINBC)
PID_CMD = 207       # PID (207 = RPM)
DATA_NUM = 7        # PID 207 DATA_NUM
WHEEL_L_ENABLE = 1  # L모터 Enable
WHEEL_R_ENABLE = 1  # R모터 Enable
DUMMY = 0           # CRC 전 DUMMY

portM = "/dev/ttyMD400" #PC에 연결된 포트명
baudM = 57600  # 시리얼 보드레이트(통신속도)
try:
    serM = serial.Serial(portM, baudM, timeout=1)
    print("Connect _ MD400T")
except:
    print("Not Connect _ MD400T _")

def speed_msg(L_SPD, R_SPD): # SPEED MSG
    MOTOR_L_SPD = [L_SPD & 255,(L_SPD >> 8)&255] # SPEED LOW
    MOTOR_R_SPD = [R_SPD & 255,(R_SPD >> 8)&255] # SPEED HI
    send_msg = [RMID, TMID, ID, PID_CMD, DATA_NUM, WHEEL_L_ENABLE, MOTOR_L_SPD[0],MOTOR_L_SPD[1],WHEEL_R_ENABLE,MOTOR_R_SPD[0],MOTOR_R_SPD[1],DUMMY]
    CHC = (~sum(send_msg)+1)&255 # CHECKSUM
    send_msg.append(CHC) # CHC 추가
    serM.write(send_msg) # MD400T 송신
    return

def stop_msg(): # STOP MSG
    return speed_msg(0,0)

def monitoring_bc(on): # MAIN DATA monitoring ON(1) / OFF(0)
    if on == 1:
        send_msg = [RMID, TMID, ID, 10, 1, 5, 128]
        return serM.write(send_msg)
    elif on == 0:
        send_msg = [RMID, TMID, ID, 10, 1, 6, 127]
        return serM.write(send_msg)
    
def data_monitor(n): # MAIN DATA Monitoring 값 추출
    return bytearray(serM.read(23))[n]

def vel_speed_msg(left, right):
    rad = 0.1375            # 바퀴 반지름
    rpm = (2*3.14*rad)      # 한바퀴 돌아갈때 거리 약 0.83m
    spd = left*60
    vel_spd_msg = int((spd/rpm)*15)
    #speed_msg(vel_spd_msg,-vel_spd_msg)
    


Connect _ MD400T


In [2]:
def vel_speed_msg(left, right):
    rad = 0.1375            # 바퀴 반지름
    rpm = (2*3.14*rad)      # 한바퀴 돌아갈때 거리 약 0.83m
    spd_L = left*60
    spd_R = right*60
    vel_spd_msg_L = int((spd_L/rpm)*15)
    vel_spd_msg_R = int((spd_R/rpm)*15)
    speed_msg(vel_spd_msg_L,-vel_spd_msg_R)
    print(vel_spd_msg_L, -vel_spd_msg_R )
    
vel_speed_msg(0.2, 0.1)

208 -104


In [11]:
stop_msg()

In [21]:
from pyjoystick.sdl2 import Key, Joystick, run_event_loop

global current_spd 


def print_add(joy):
    return

def print_remove(joy):
    print('Removed', joy)

def key_received(key):
    global current_spd
    if key.value == Key.HAT_UP:
        if current_spd >= 2:
            current_spd = 2
        else:
            current_spd += 0.1
        print("UP", current_spd)
    elif key.value == Key.HAT_DOWN:
        if current_spd <= -2:
            current_spd = -2
        else:
            current_spd -= 0.1
        print("DOWN", current_spd)
    elif key.value == Key.HAT_CENTERED:
        vel_speed_msg(current_spd, current_spd)
        print("CENTER", current_spd)
    elif key.value == Key.HAT_LEFT:
        vel_speed_msg(current_spd, current_spd+0.2)
        print("LEFT", current_spd)
    elif key.value == Key.HAT_RIGHT:
        vel_speed_msg(current_spd+0.2, current_spd)
        print("RIGHT", current_spd)

run_event_loop(print_add, print_remove, key_received)

UP 0.1
UP 0.2
DOWN 0.1
104 -104
CENTER 0.1
UP 0.2
208 -208
CENTER 0.2
DOWN 0.1
104 -104
CENTER 0.1
DOWN 0.0
0 0
CENTER 0.0
UP 0.1
104 -104
CENTER 0.1
312 -104
RIGHT 0.1
104 -104
CENTER 0.1
312 -104
RIGHT 0.1
104 -104
CENTER 0.1
UP 0.2
208 -208
CENTER 0.2
UP 0.30000000000000004
312 -312
CENTER 0.30000000000000004
UP 0.4
416 -416
CENTER 0.4
UP 0.5
521 -521
CENTER 0.5
UP 0.6
625 -625
CENTER 0.6
UP 0.7
729 -729
CENTER 0.7
UP 0.7999999999999999
833 -833
CENTER 0.7999999999999999
833 -1042
LEFT 0.7999999999999999
833 -833
CENTER 0.7999999999999999
833 -1042
LEFT 0.7999999999999999
833 -833
CENTER 0.7999999999999999
833 -1042
LEFT 0.7999999999999999
833 -833
CENTER 0.7999999999999999
DOWN 0.7
729 -729
CENTER 0.7
DOWN 0.6
625 -625
CENTER 0.6
DOWN 0.5
521 -521
CENTER 0.5
DOWN 0.4
416 -416
CENTER 0.4
DOWN 0.30000000000000004
312 -312
CENTER 0.30000000000000004
521 -312
RIGHT 0.30000000000000004
312 -312
CENTER 0.30000000000000004
521 -312
RIGHT 0.30000000000000004
312 -312
CENTER 0.3000000000000

In [1]:
# from pyjoystick.sdl2 import Key, Joystick, run_event_loop
# from zmq import PUSH

# stop_msg()

# def print_add(joy):
#     print('Added', joy)

# def print_remove(joy):
#     print('Removed', joy)

# def key_received(key):
#     global current_spd
#     print('received', key)
#     if key.value == Key.HAT_UP:
#         current_spd += 0.1
#         print(current_spd)
#         vel_speed_msg(current_spd, current_spd)
#     elif key.value == Key.HAT_DOWN:
#         if current_spd <= 0.12:
#             current_spd = 0
#         else:
#             current_spd -= 0.1
#         vel_speed_msg(current_spd, current_spd)
#         print(current_spd)
#         print('down')
#     if key.value == Key.HAT_LEFT:
#         vel_speed_msg(current_spd, current_spd+0.2)
#         print('left')
#     elif key.value == Key.HAT_UPLEFT:
#         print('upleft')
#     elif key.value == Key.HAT_DOWNLEFT:
#         print('downleft')
#     elif key.value == Key.HAT_RIGHT:
#         vel_speed_msg(current_spd+0.2, current_spd)
#         print('right')
#     elif key.value == Key.HAT_UPRIGHT:
#         print('upright')
#     elif key.value == Key.HAT_DOWNRIGHT:
#         print("downright")
#     elif key.value == Key.HAT_CENTERED:
#         vel_speed_msg(current_spd, current_spd)
#         print("CENTER")
#     elif key.value == Key.BUTTON:        
#         print('Button1')

# run_event_loop(print_add, print_remove, key_received)